In [1]:
# !pip install coiled

In [2]:
# !coiled login --token $coiled_token

Traceback (most recent call last):
  File "c:\users\argenisleon\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\argenisleon\anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\argenisleon\Anaconda3\Scripts\coiled.exe\__main__.py", line 7, in <module>
  File "c:\users\argenisleon\anaconda3\lib\site-packages\click\core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "c:\users\argenisleon\anaconda3\lib\site-packages\click\core.py", line 782, in main
    rv = self.invoke(ctx)
  File "c:\users\argenisleon\anaconda3\lib\site-packages\click\core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "c:\users\argenisleon\anaconda3\lib\site-packages\click\core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "c:\users\argenisleon\anaconda3\lib\site-packages\click\core.py", line 610, in invoke
    return cal

In [3]:
import coiled

In [ ]:
coiled.create_software_environment(name='zarr', conda=['xarray', 'zarr','dask=2.2.23','distributed'])

In [ ]:
coiled.create_cluster_configuration(
    name="bumblebee",
    worker_memory="16 GiB",
    worker_cpu=4,
    scheduler_memory="4 GiB",
    scheduler_cpu=1,
    software="coiled/default"
)

In [ ]:
cluster.name #salvar en bumblebee

In [ ]:
coiled.list_clusters()

In [ ]:
cluster = coiled.Cluster(configuration='bumblebee')

In [ ]:
cluster.name # argenisleon-xxxxxxxx-x

In [ ]:
coiled.list_clusters()  # {"argenisleon-xxxxxxxx-x":{...}}

In [ ]:
from dask.distributed import Client
client = Client(name=cluster.name)
print('Dashboard:', client.dashboard_link)

In [ ]:
dir(coiled)

In [ ]:
import dask.dataframe as dd

df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
).persist()

df.groupby("passenger_count").tip_amount.mean().compute()

In [1]:
op

NameError: name 'op' is not defined